In [1]:
import pymongo
from pymongo import MongoClient
import json
import tweepy
import twitter
from pprint import pprint
import configparser

In [2]:
config = configparser.ConfigParser()
config.read('config.ini')

CONSUMER_KEY      = config['mytwitter']['api_key']
CONSUMER_SECRET   = config['mytwitter']['api_secrete']
OAUTH_TOKEN       = config['mytwitter']['access_token']
OATH_TOKEN_SECRET = config['mytwitter']['access_secrete']

mongod_connect = config['mymongo']['connection']

In [3]:
client = MongoClient(mongod_connect)
db = client.tweet_db # create a database named tweet_db
tweet_collection = db.tweet_collection #create a collection called tweet_collection
tweet_collection.create_index([("id", pymongo.ASCENDING)],unique = True) # make sure the collected tweets are unique

'id_1'

In [4]:
stream_auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
stream_auth.set_access_token(OAUTH_TOKEN, OATH_TOKEN_SECRET)

strem_api = tweepy.API(stream_auth)

In [5]:
track = ['jmu'] # define the keywords, tweets contain JMU

locations = [-78.9326449,38.4150904,-78.8816972,38.4450731] #defin the location, in Harrisonburg, VA

In [9]:
class MyStreamListener(tweepy.StreamListener):
    def on_status(self, status):
        print (status.id_str)
        try:
            tweet_collection.insert_one(status._json)
        except:
            pass
  
    def on_error(self, status_code):
        if status_code == 420:
            #returning False in on_data disconnects the stream
            return False
myStreamListener = MyStreamListener()
myStream = tweepy.Stream(auth = strem_api.auth, listener=myStreamListener)
myStream.filter(track=track, locations = locations)

1191800400090939392


KeyboardInterrupt: 

In [10]:
rest_auth = twitter.oauth.OAuth(OAUTH_TOKEN,OATH_TOKEN_SECRET,CONSUMER_KEY,CONSUMER_SECRET)
rest_api = twitter.Twitter(auth=rest_auth)

In [11]:
count = 100 #number of returned tweets, default and max is 100
geocode = "38.4392897,-78.9412224,50mi"  # defin the location, in Harrisonburg, VA
q = "jmu"                               #define the keywords, tweets contain JMU

In [12]:
search_results = rest_api.search.tweets( count=count,q=q, geocode=geocode)
statuses = search_results["statuses"]
since_id_new = statuses[-1]['id']
for statuse in statuses:
    try:
        tweet_collection.insert_one(statuse)
        pprint(statuse['created_at'])# print the date of the collected tweets
    except:
        pass

'Tue Nov 05 19:27:56 +0000 2019'
'Tue Nov 05 19:12:09 +0000 2019'
'Tue Nov 05 19:07:09 +0000 2019'
'Tue Nov 05 19:07:08 +0000 2019'
'Tue Nov 05 19:06:36 +0000 2019'
'Tue Nov 05 19:04:32 +0000 2019'
'Tue Nov 05 19:04:06 +0000 2019'
'Tue Nov 05 19:03:38 +0000 2019'
'Tue Nov 05 18:51:23 +0000 2019'
'Tue Nov 05 18:50:37 +0000 2019'
'Tue Nov 05 18:33:14 +0000 2019'
'Tue Nov 05 18:20:34 +0000 2019'
'Tue Nov 05 18:10:29 +0000 2019'
'Tue Nov 05 18:09:27 +0000 2019'
'Tue Nov 05 18:04:09 +0000 2019'
'Tue Nov 05 18:03:18 +0000 2019'
'Tue Nov 05 18:01:18 +0000 2019'
'Tue Nov 05 18:00:46 +0000 2019'
'Tue Nov 05 17:58:49 +0000 2019'
'Tue Nov 05 17:57:28 +0000 2019'
'Tue Nov 05 17:34:27 +0000 2019'
'Tue Nov 05 17:32:17 +0000 2019'
'Tue Nov 05 17:04:10 +0000 2019'
'Tue Nov 05 16:57:54 +0000 2019'
'Tue Nov 05 16:49:24 +0000 2019'
'Tue Nov 05 16:49:06 +0000 2019'
'Tue Nov 05 16:48:37 +0000 2019'
'Tue Nov 05 16:45:03 +0000 2019'
'Tue Nov 05 16:19:33 +0000 2019'
'Tue Nov 05 16:18:52 +0000 2019'
'Tue Nov 0

In [13]:
since_id_old = 0
while(since_id_new != since_id_old):
    since_id_old = since_id_new
    search_results = rest_api.search.tweets( count=count,q=q,
                        geocode=geocode, max_id= since_id_new)
    statuses = search_results["statuses"]
    since_id_new = statuses[-1]['id']
    for statuse in statuses:
        try:
            tweet_collection.insert_one(statuse)
            pprint(statuse['created_at']) # print the date of the collected tweets
        except:
            pass

'Tue Nov 05 05:00:58 +0000 2019'
'Tue Nov 05 04:49:54 +0000 2019'
'Tue Nov 05 04:11:08 +0000 2019'
'Tue Nov 05 04:05:48 +0000 2019'
'Tue Nov 05 04:04:34 +0000 2019'
'Tue Nov 05 04:03:39 +0000 2019'
'Tue Nov 05 03:37:18 +0000 2019'
'Tue Nov 05 03:34:21 +0000 2019'
'Tue Nov 05 03:15:03 +0000 2019'
'Tue Nov 05 03:13:31 +0000 2019'
'Tue Nov 05 03:13:08 +0000 2019'
'Tue Nov 05 03:01:35 +0000 2019'
'Tue Nov 05 02:55:20 +0000 2019'
'Tue Nov 05 02:54:53 +0000 2019'
'Tue Nov 05 02:50:25 +0000 2019'
'Tue Nov 05 02:48:54 +0000 2019'
'Tue Nov 05 02:48:46 +0000 2019'
'Tue Nov 05 02:31:21 +0000 2019'
'Tue Nov 05 02:14:44 +0000 2019'
'Tue Nov 05 01:52:35 +0000 2019'
'Tue Nov 05 01:00:31 +0000 2019'
'Tue Nov 05 00:53:17 +0000 2019'
'Tue Nov 05 00:25:26 +0000 2019'
'Tue Nov 05 00:18:17 +0000 2019'
'Tue Nov 05 00:09:58 +0000 2019'
'Mon Nov 04 23:55:26 +0000 2019'
'Mon Nov 04 23:51:07 +0000 2019'
'Mon Nov 04 23:46:01 +0000 2019'
'Mon Nov 04 23:39:29 +0000 2019'
'Mon Nov 04 23:38:24 +0000 2019'
'Mon Nov 0

'Sun Nov 03 00:43:36 +0000 2019'
'Sun Nov 03 00:42:27 +0000 2019'
'Sun Nov 03 00:39:50 +0000 2019'
'Sun Nov 03 00:37:50 +0000 2019'
'Sun Nov 03 00:36:48 +0000 2019'
'Sun Nov 03 00:34:01 +0000 2019'
'Sun Nov 03 00:33:01 +0000 2019'
'Sun Nov 03 00:32:23 +0000 2019'
'Sun Nov 03 00:27:13 +0000 2019'
'Sun Nov 03 00:24:10 +0000 2019'
'Sun Nov 03 00:19:12 +0000 2019'
'Sun Nov 03 00:16:57 +0000 2019'
'Sun Nov 03 00:14:40 +0000 2019'
'Sun Nov 03 00:07:57 +0000 2019'
'Sat Nov 02 23:50:52 +0000 2019'
'Sat Nov 02 23:47:55 +0000 2019'
'Sat Nov 02 23:43:24 +0000 2019'
'Sat Nov 02 23:43:10 +0000 2019'
'Sat Nov 02 23:43:10 +0000 2019'
'Sat Nov 02 23:42:56 +0000 2019'
'Sat Nov 02 23:42:33 +0000 2019'
'Sat Nov 02 23:39:23 +0000 2019'
'Sat Nov 02 23:38:24 +0000 2019'
'Sat Nov 02 23:26:36 +0000 2019'
'Sat Nov 02 23:25:07 +0000 2019'
'Sat Nov 02 23:08:51 +0000 2019'
'Sat Nov 02 23:08:04 +0000 2019'
'Sat Nov 02 23:01:55 +0000 2019'
'Sat Nov 02 22:58:10 +0000 2019'
'Sat Nov 02 22:53:39 +0000 2019'
'Sat Nov 0

'Fri Nov 01 19:13:51 +0000 2019'
'Fri Nov 01 19:13:02 +0000 2019'
'Fri Nov 01 19:12:37 +0000 2019'
'Fri Nov 01 19:12:09 +0000 2019'
'Fri Nov 01 19:11:07 +0000 2019'
'Fri Nov 01 19:07:17 +0000 2019'
'Fri Nov 01 19:07:10 +0000 2019'
'Fri Nov 01 19:05:00 +0000 2019'
'Fri Nov 01 19:04:54 +0000 2019'
'Fri Nov 01 19:03:39 +0000 2019'
'Fri Nov 01 19:02:02 +0000 2019'
'Fri Nov 01 19:01:42 +0000 2019'
'Fri Nov 01 19:00:46 +0000 2019'
'Fri Nov 01 19:00:00 +0000 2019'
'Fri Nov 01 18:59:23 +0000 2019'
'Fri Nov 01 18:56:20 +0000 2019'
'Fri Nov 01 18:55:31 +0000 2019'
'Fri Nov 01 18:52:34 +0000 2019'
'Fri Nov 01 18:48:45 +0000 2019'
'Fri Nov 01 18:48:06 +0000 2019'
'Fri Nov 01 18:47:23 +0000 2019'
'Fri Nov 01 18:45:04 +0000 2019'
'Fri Nov 01 18:41:37 +0000 2019'
'Fri Nov 01 18:36:18 +0000 2019'
'Fri Nov 01 18:35:35 +0000 2019'
'Fri Nov 01 18:31:48 +0000 2019'
'Fri Nov 01 18:27:52 +0000 2019'
'Fri Nov 01 18:19:12 +0000 2019'
'Fri Nov 01 18:17:35 +0000 2019'
'Fri Nov 01 18:17:22 +0000 2019'
'Fri Nov 0

'Thu Oct 31 19:02:45 +0000 2019'
'Thu Oct 31 19:02:43 +0000 2019'
'Thu Oct 31 19:02:06 +0000 2019'
'Thu Oct 31 19:01:59 +0000 2019'
'Thu Oct 31 19:01:57 +0000 2019'
'Thu Oct 31 19:01:47 +0000 2019'
'Thu Oct 31 19:01:43 +0000 2019'
'Thu Oct 31 19:01:28 +0000 2019'
'Thu Oct 31 19:00:33 +0000 2019'
'Thu Oct 31 19:00:14 +0000 2019'
'Thu Oct 31 18:57:32 +0000 2019'
'Thu Oct 31 18:54:34 +0000 2019'
'Thu Oct 31 18:53:41 +0000 2019'
'Thu Oct 31 18:51:43 +0000 2019'
'Thu Oct 31 18:51:04 +0000 2019'
'Thu Oct 31 18:50:51 +0000 2019'
'Thu Oct 31 18:50:04 +0000 2019'
'Thu Oct 31 18:50:02 +0000 2019'
'Thu Oct 31 18:49:41 +0000 2019'
'Thu Oct 31 18:45:06 +0000 2019'
'Thu Oct 31 18:45:00 +0000 2019'
'Thu Oct 31 18:32:49 +0000 2019'
'Thu Oct 31 18:28:36 +0000 2019'
'Thu Oct 31 18:27:08 +0000 2019'
'Thu Oct 31 18:24:10 +0000 2019'
'Thu Oct 31 18:22:17 +0000 2019'
'Thu Oct 31 18:22:14 +0000 2019'
'Thu Oct 31 18:21:41 +0000 2019'
'Thu Oct 31 18:21:29 +0000 2019'
'Thu Oct 31 18:06:44 +0000 2019'
'Thu Oct 3

'Wed Oct 30 10:17:46 +0000 2019'
'Wed Oct 30 08:51:57 +0000 2019'
'Wed Oct 30 08:36:29 +0000 2019'
'Wed Oct 30 04:53:36 +0000 2019'
'Wed Oct 30 04:53:36 +0000 2019'
'Wed Oct 30 04:39:06 +0000 2019'
'Wed Oct 30 04:08:23 +0000 2019'
'Wed Oct 30 03:46:57 +0000 2019'
'Wed Oct 30 03:21:59 +0000 2019'
'Wed Oct 30 03:13:01 +0000 2019'
'Wed Oct 30 03:04:06 +0000 2019'
'Wed Oct 30 02:53:32 +0000 2019'
'Wed Oct 30 02:52:56 +0000 2019'
'Wed Oct 30 02:52:40 +0000 2019'
'Wed Oct 30 02:35:50 +0000 2019'
'Wed Oct 30 02:18:39 +0000 2019'
'Wed Oct 30 02:11:39 +0000 2019'
'Wed Oct 30 02:10:45 +0000 2019'
'Wed Oct 30 02:08:45 +0000 2019'
'Wed Oct 30 02:02:32 +0000 2019'
'Wed Oct 30 01:57:37 +0000 2019'
'Wed Oct 30 01:55:24 +0000 2019'
'Wed Oct 30 01:52:34 +0000 2019'
'Wed Oct 30 01:48:18 +0000 2019'
'Wed Oct 30 01:46:34 +0000 2019'
'Wed Oct 30 01:28:59 +0000 2019'
'Wed Oct 30 01:22:26 +0000 2019'
'Wed Oct 30 01:18:49 +0000 2019'
'Wed Oct 30 01:14:50 +0000 2019'
'Wed Oct 30 01:11:24 +0000 2019'
'Wed Oct 3

'Mon Oct 28 17:02:07 +0000 2019'
'Mon Oct 28 17:01:00 +0000 2019'
'Mon Oct 28 16:48:07 +0000 2019'
'Mon Oct 28 16:45:53 +0000 2019'
'Mon Oct 28 16:44:54 +0000 2019'
'Mon Oct 28 16:36:00 +0000 2019'
'Mon Oct 28 16:27:27 +0000 2019'
'Mon Oct 28 16:26:38 +0000 2019'
'Mon Oct 28 16:16:22 +0000 2019'
'Mon Oct 28 16:15:00 +0000 2019'
'Mon Oct 28 16:12:41 +0000 2019'
'Mon Oct 28 16:06:14 +0000 2019'
'Mon Oct 28 16:06:11 +0000 2019'
'Mon Oct 28 16:03:42 +0000 2019'
'Mon Oct 28 16:03:28 +0000 2019'
'Mon Oct 28 16:02:24 +0000 2019'
'Mon Oct 28 15:58:12 +0000 2019'
'Mon Oct 28 15:56:38 +0000 2019'
'Mon Oct 28 15:56:35 +0000 2019'
'Mon Oct 28 15:56:33 +0000 2019'
'Mon Oct 28 15:56:32 +0000 2019'
'Mon Oct 28 15:52:11 +0000 2019'
'Mon Oct 28 15:50:34 +0000 2019'
'Mon Oct 28 15:45:14 +0000 2019'
'Mon Oct 28 15:44:18 +0000 2019'
'Mon Oct 28 15:43:57 +0000 2019'
'Mon Oct 28 15:43:36 +0000 2019'
'Mon Oct 28 15:43:23 +0000 2019'
'Mon Oct 28 15:42:43 +0000 2019'
'Mon Oct 28 15:42:41 +0000 2019'
'Mon Oct 2

In [14]:
print(tweet_collection.estimated_document_count())# number of tweets collected

user_cursor = tweet_collection.distinct("user.id")
print (len(user_cursor)) # number of unique Twitter users

1647
838


In [15]:
tweet_collection.create_index([("text", pymongo.TEXT)], name='text_index', default_language='english') # create a text index

tweet_cursor = tweet_collection.find({"$text": {"$search": "football"}}) # return tweets contain football
for document in tweet_cursor:
    try:
        print ('----')
#         pprint (document) # use pprint to print the entire tweet document
   
        print ('name:', document["user"]["name"]) # user name
        print ('text:', document["text"])         # tweets
    except:
        print ("***error in encoding")
        pass

----
name: JMU Sports News
text: Good read on JMU football ⬇️ https://t.co/iQkCHacqlo
----
name: 98RockVirginia
text: JMU Football to play VA TECH in 2025  https://t.co/QVfbO1eFTV
----
name: Greg Madia
text: Any JMU football questions for the bye week edition of The Walkthrough Podcast? Drop them here and I'll answer on the pod.
----
name: Nick Lorensen
text: Stopped in Harrisonburg to get some Cook-Out.

JMU’s football stadium is much bigger than I thought it would be
----
name: James Faris
text: RT @JMUSportsNews: Good read on JMU football ⬇️ https://t.co/iQkCHacqlo
----
name: Jeremy Hunt
text: RT @Madia_DNRSports: Any JMU football questions for the bye week edition of The Walkthrough Podcast? Drop them here and I'll answer on the…
----
name: Michael Evangelista
text: RT @Madia_DNRSports: Any JMU football questions for the bye week edition of The Walkthrough Podcast? Drop them here and I'll answer on the…
----
name: ❤️ #ModArt 🖼️
text: RT @98rockme: JMU Football to play VA TECH in 20